In [1]:
%load_ext autoreload
%autoreload 2
from fun import *

In [2]:
db_filename = '/home/msarrias/dbs/homo_1.db'
db = gffutils.FeatureDB(db_filename, keep_order=True)
fasta_GRCh38_seq = SeqIO.parse(open('/home/msarrias/data/Homo_sapiens.GRCh38.dna.chromosome.1.fa'),'fasta')
fasta_GRCh38_seq = { fasta.id : {'+' : str(fasta.seq),
                                 '-': str(fasta.seq.reverse_complement())} for fasta in fasta_GRCh38_seq}

gene_hierarchy_dict = generate_gene_hierarchy_dict(db)
gene_transcript_dict, gene_hierarchy_dict_with_ce = get_gene_hierarchy_dict_with_coding_exons(gene_hierarchy_dict)

In [3]:
random.seed(10)
max_n_exons = 10
sample_genes, simulated_genes, sample_transcripts_dict = {}, {}, {}
copy_gene_hierarchy_dict_with_ce = copy.deepcopy(gene_hierarchy_dict_with_ce)
settings_list = sorted([(n_exons, np.random.randint(2, n_exons-1))
                        for n_exons in random.sample(range(4,max_n_exons+1), 7)],
                       key = lambda item:item[0])

In [ ]:
for setting in settings_list:
    n_exons, exon_dup = setting
    sample_genes[setting] = {}
    sample_transcripts_dict[setting] = {}
    for gene_id, gene_hierarchy_dict_ in copy_gene_hierarchy_dict_with_ce.items():
        if db[gene_id].strand == '+' and len(gene_hierarchy_dict_) > 0:
            transcript = random.choice(list(gene_hierarchy_dict_.keys()))
            transcript_temp = {}
            trapt_hier_dict = copy.deepcopy(gene_hierarchy_dict_[transcript])
            n_coding_exons = [i['type'] for i in trapt_hier_dict.values() if i['type'] == 'coding_exon']
            if len(n_coding_exons) == n_exons:
                sample_genes[setting][gene_id] = copy.deepcopy(trapt_hier_dict)
                sample_transcripts_dict[setting][gene_id] = transcript
            if len(sample_genes[setting].keys()) > 19:
                break
    simulated_genes[setting] = {}
    for pos in ['before', 'after']:
        sim_genes_sample = simulate_genes_with_coding_exons_duplications(exon_dup, 
                                                                          pos, 
                                                                          sample_genes[setting],
                                                                          gene_hierarchy_dict_with_ce,
                                                                          fasta_GRCh38_seq['1']['+'],
                                                                          db)
        sorted_gene_sample = {gene[0]: copy.deepcopy(sim_genes_sample[gene[0]]) 
                              for gene in sorted([(key, value['coord']) 
                                                  for key, value in sim_genes_sample.items()],
                                                 key = lambda item: (item[1].lower, item[1].upper))
                     }
        simulated_genes[setting][pos] = sorted_gene_sample
        new_seq, len_dup = generate_new_genome_seq(sorted_gene_sample,
                                                   gene_hierarchy_dict_with_ce,
                                                   fasta_GRCh38_seq['1']['+'], db)
        print(len(new_seq) - len_dup == len(fasta_GRCh38_seq['1']['+']))
        coding_exons_interv = {gene_id :
                               {i:j for i, j in zip(transcr_dict['features_order'],
                                                    transcr_dict['features_intervals']) if 'exon' in i} 
                               for gene_id, transcr_dict in sorted_gene_sample.items() 
                              }
        sim_genes_interv_in_chr = get_simulated_new_intervals(fasta_GRCh38_seq['1']['+'],
                                                              new_seq,
                                                              sorted_gene_sample,
                                                              coding_exons_interv)
        print(setting, pos)
        for gene_id, gene_dict in sim_genes_interv_in_chr.items():
            for id_, annot in gene_dict['coding_exons_dups'].items():
                if len(annot) == 2:
                    s, e = annot[0]
                    print(gene_id, 'split intron ends with: ', new_seq[(s-2):(s)],
                          '   ', 'starts with: ', new_seq[e:(e+2)])
        print('    ')            
        with open('../files/GRCh38_dup_' + str(n_exons)+ '_' + str(exon_dup) + '_' + pos + '.pkl', 'wb') as handle:
            pickle.dump(sim_genes_interv_in_chr, handle)

        record = SeqRecord(
            Seq(new_seq),
            id = '+')
        with open("GRCh38_dup_" + str(n_exons)+ '_' + str(exon_dup) + '_' + pos + ".fa", "w") as handle:
            SeqIO.write(record, handle, "fasta")

True
(4, 2) before
gene:ENSG00000067606 split intron ends with:  AG     starts with:  GT
gene:ENSG00000162592 split intron ends with:  AG     starts with:  GT
gene:ENSG00000173662 split intron ends with:  AG     starts with:  GT
gene:ENSG00000116273 split intron ends with:  AG     starts with:  GT
gene:ENSG00000041988 split intron ends with:  AG     starts with:  GT
gene:ENSG00000171735 split intron ends with:  AG     starts with:  GT
gene:ENSG00000162444 split intron ends with:  AG     starts with:  GT
gene:ENSG00000116663 split intron ends with:  AG     starts with:  GT
gene:ENSG00000204518 split intron ends with:  AG     starts with:  GT
gene:ENSG00000188984 split intron ends with:  AG     starts with:  GT
gene:ENSG00000065526 split intron ends with:  AG     starts with:  GT
gene:ENSG00000159339 split intron ends with:  AG     starts with:  GT
gene:ENSG00000159339 split intron ends with:  AG     starts with:  TC
gene:ENSG00000270136 split intron ends with:  AG     starts with:  GT
g

True
(6, 2) after
gene:ENSG00000169962 split intron ends with:  AG     starts with:  GT
gene:ENSG00000169885 split intron ends with:  AG     starts with:  GT
gene:ENSG00000157916 split intron ends with:  AG     starts with:  GT
gene:ENSG00000157873 split intron ends with:  AG     starts with:  GT
gene:ENSG00000157873 split intron ends with:  AG     starts with:  GT
gene:ENSG00000116288 split intron ends with:  AG     starts with:  GT
gene:ENSG00000142655 split intron ends with:  AG     starts with:  GT
gene:ENSG00000162490 split intron ends with:  AG     starts with:  GT
gene:ENSG00000162460 split intron ends with:  AG     starts with:  GT
gene:ENSG00000169504 split intron ends with:  AG     starts with:  GT
gene:ENSG00000183726 split intron ends with:  AG     starts with:  GT
gene:ENSG00000198830 split intron ends with:  CC     starts with:  GC
gene:ENSG00000198830 split intron ends with:  AG     starts with:  GT
gene:ENSG00000121766 split intron ends with:  AG     starts with:  GT
ge

True


In [ ]:
human_protein_seqs = {}
fasta_sequences = SeqIO.parse(open("fasta_files/Homo_sapiens.GRCh38.pep.all.fa"),'fasta')
for fasta in fasta_sequences:
    gene = [i for i in fasta.description.rsplit(' ') if 'gene:' in i]
    if gene:
        if gene[0] not in human_protein_seqs:
            human_protein_seqs[gene[0]] = {}
        human_protein_seqs[gene[0]][fasta.id] = str(fasta.seq)
        
for setting, setting_dict in sample_genes.items():
    genes_list = list(setting_dict.keys())
    proteins_simulated_genes = {}
    protein_gene_dict = {}
    with open("fasta_files/prot_seq_" +str(setting[0]) + '_' + str(setting[1]) + ".fa", "w") as handle:
        for gene in genes_list:
            gene_id_prot_dict = [i for i in list(human_protein_seqs.keys()) if gene in i]
            if gene_id_prot_dict:
                temp = copy.deepcopy(human_protein_seqs[gene_id_prot_dict[0]])
                prot_random = random.choice(list(temp.keys()))
                protein_gene_dict[prot_random.split('.')[0]] = gene_id_prot_dict[0].split('.')[0]
                record = SeqRecord(Seq(temp[prot_random]),
                                   id = str(prot_random),
                                   description= gene_id_prot_dict[0])
                SeqIO.write(record, handle, "fasta")
                proteins_simulated_genes[gene_id_prot_dict[0]] = temp[prot_random]
    with open("../files/protein_gene_corresp_" + str(setting[0]) + '_' + str(setting[1]) + ".pkl", 'wb') as handle:
        pickle.dump(protein_gene_dict, handle)

In [ ]:
with open("../files/transcripts_dict_.pkl", 'wb') as handle:
    pickle.dump({'simulated_genes':simulated_genes,
                 'sample_genes':sample_genes,
                 'sample_transcripts_dict':sample_transcripts_dict},
                handle)

## Exonerate

In [32]:
for setting in settings_list[2:]:
    queries = "fasta_files/prot_seq_" +str(setting[0]) + '_' + str(setting[1]) + ".fa"
    for pos in ['before', 'after']:
        hit = "fasta_files/GRCh38_dup_" + str(setting[0]) + '_' + str(setting[1]) +'_' + pos +  ".fa"
        out_file = 'Exonerate/exonerate_' + str(setting[0]) + '_' + str(setting[1]) +'_' + pos + '.txt'
        f = open(out_file, "w")
        subprocess.call(["exonerate", "--query", queries, '--target',  hit, '--model', 'protein2genome'], stdout=f)


## Miniprot

In [33]:
for setting in settings_list[2:]:
    queries = "fasta_files/prot_seq" +str(setting[0]) + '_' + str(setting[1]) + ".fa"
    for pos in ['before', 'after']:
        hit = "fasta_files/GRCh38_with_ce_dup_" + str(setting[0]) + '_' + str(setting[1]) +'_' + pos +  ".fa"
        out_file = 'Miniprot/miniprot_' + str(setting[0]) + '_' + str(setting[1]) + pos + '.txt'
        f = open(out_file, "w")
        subprocess.call(["/domus/h1/msarrias/bin/miniprot/miniprot", "-t8", hit, queries], stdout=f)


[M::mp_ntseq_read@1.254*1.02] read 248883353 bases in 1 contigs
[M::mp_idx_build@1.254*1.02] 1944402 blocks
[M::mp_idx_build@17.280*0.99] collected syncmers
[M::mp_idx_build@18.023*0.99] 70569585 kmer-block pairs
[M::mp_idx_print_stat] 1101931 distinct k-mers; mean occ of infrequent k-mers: 64.04; 0 frequent k-mers accounting for 0 occurrences
[M::worker_pipeline::19.029*0.99] mapped 40 sequences
[M::main] Version: 0.1-r97
[M::main] CMD: /domus/h1/msarrias/bin/miniprot/miniprot -t8 fasta_files/GRCh38_with_ce_dup_6_2_before.fa fasta_files/prot_seq6_2.fa
[M::main] Real time: 19.075 sec; CPU: 18.908 sec; Peak RSS: 1.599 GB
[M::mp_ntseq_read@1.255*1.02] read 248883353 bases in 1 contigs
[M::mp_idx_build@1.255*1.02] 1944402 blocks
[M::mp_idx_build@17.124*0.99] collected syncmers
[M::mp_idx_build@17.827*0.99] 70569607 kmer-block pairs
[M::mp_idx_print_stat] 1101931 distinct k-mers; mean occ of infrequent k-mers: 64.04; 0 frequent k-mers accounting for 0 occurrences
[M::worker_pipeline::18.81